# Tools

## Data preprocessing

In [176]:
%load_ext autoreload
%autoreload 2
import os
import random
import sqlite3 
import numpy as np
import json
import math
from tqdm.notebook import tqdm
from tqdm import tqdm
tqdm.pandas()

import calendar
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

import Advan_operator as ad_op  

pd.set_option('display.max_columns', None)

import logging
# Create a logger
logger_name = 'all_logger'
logger = logging.getLogger(logger_name)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load hourly population

#### Asign file locations

In [177]:
# data_dir = r'F:\SafeGraph\Advan_2023_API\Neighborhood_Patterns'   # Lenovo

# Dell desktop
# save_dir = r'D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test_2024_home_panel_2018\visualization'  
# data_dir = r'D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test_2024_home_panel_2018'  # Dell desktop
# ACS_file = r"D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_data\Safegraph_bias\cbg_acs_2019_county_tract_new20230929_cleaned.csv"
 

# data_dir = r'K:\SafeGraph\Advan_2023_API\Neighborhood_Patterns'   # Dell 
# save_dir = r'D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test'  # Dell
# ACS_file = r"D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_data\Safegraph_bias\cbg_acs_2019_county_tract_new20230929_cleaned.csv"
home_panel_fname = f"D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\Monthly_Patterns_home_panel_summary_2019_2023.zip"
# data_dir = r'F:\SafeGraph\Advan_2023_API\Neighborhood_Patterns'   # Lenovo 
# save_dir = r'D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test'  # Lenovo
# ACS_file = r"F:\Research\Wild_fire\cbg_acs_2019_county_tract_new20230929_cleaned.csv"
# home_panel_fname = f"F:\SafeGraph\Advan_2023_API\Monthly_Patterns_home_panel_summary\Monthly_Patterns_home_panel_summary_2019_2023.csv"
# CBG_2019_fname = r"D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\blockgoups2019.zip"


# # 2018 desktop
save_dir = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test_2024_home_panel_2018\visualization'  # 2018 desktop
data_dir = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test_2024_home_panel_2018'  # Dell desktop
# save_dir = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test_2023_home_panel\visualization'  # 2018 desktop
# data_dir = r'E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test_2023_home_panel'  # Dell desktop

ACS_file = r"E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_data\Safegraph_bias\cbg_acs_2019_county_tract_new20230929_cleaned.csv"
NYC_MTA_fname = r"E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\MTA_Daily_Ridership_Data__Beginning_2020_20240129.csv"


os.makedirs(save_dir, exist_ok=True)
save_file = True

year = 2023
month = 1

 
month_dir = os.path.join(data_dir, str(year), f'{month:02}', '01')
no_dict_colum_fname = os.path.join(save_dir, f"NP_dict_column_{year}_{month:02}.csv")
device_home_areas_fname = os.path.join(save_dir, f"NP_split_device_home_areas_{year}_{month:02}.csv")
adjusted_dwelling_time_stop_CBG_fname     = os.path.join(save_dir, f"adjusted_dwelling_time_stop_CBG_{year}_{month:02}.csv")
adjusted_dwelling_time_stop_natinol_fname = os.path.join(save_dir, f"adjusted_dwelling_time_stop_natinol_{year}_{month:02}.csv")
person_per_device_fname = os.path.join(save_dir, f"person_per_device_{year}_{month:02}.csv")
restored_CBG_hourly_vistor_count_fname = os.path.join(save_dir, f"restored_hourly_vistor_count_CBG_{year}_{month:02}.csv")
restored_target_hourly_vistor_count_fname = os.path.join(save_dir, f"restored_hourly_vistor_count_target_{year}_{month:02}.csv")
CBG_device_stop_destination_distribution_fname  = os.path.join(save_dir, f"CBG_device_stop_destination_distribution_{year}_{month:02}.csv")
CBG_device_stop_origin_distribution_fname = os.path.join(save_dir, f"CBG_device_stop_origin_distribution_{year}_{month:02}.csv")
IPF_results_fname = os.path.join(data_dir, f"IPF_origin_outgoer_hourly_{year}{month:02}.csv")
CBG_hourly_population_fname = os.path.join(data_dir, f"CBG_population_hourly_{year}{month:02}.csv")


CBG_origin_outbound_distribution_sampling_rate_fname = os.path.join(data_dir, f"CBG_origin_outbound_distribution_sampling_rate_{year}{month:02}.csv")
CBG_destination_inbound_per_device_fname = os.path.join(data_dir, f"CBG_destination_inbound_per_device_{year}{month:02}.csv")

CBG_destination_inbound_fname = os.path.join(data_dir, f"CBG_destination_inbound_{year}{month:02}.csv")
inbound_hourly_df_fname = os.path.join(data_dir, f"inbound_hourly_{year}{month:02}.csv")
CBG_agregated_inbound_fname =  os.path.join(save_dir, f"CBG_agregated_inbound_{year}{month:02}.csv")
hourly_agregated_inbound_fname =  os.path.join(save_dir, f"hourly_agregated_inbound_{year}{month:02}.csv")

 
# month_dir = r'K:\SafeGraph\Advan_2023_API\Neighborhood_Patterns\2023\06\01'
# CBG_list_fname = r"D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\ACS_2019_CBG_list.csv"

sqlite_fname = r'F:\Research\Wild_fire\Neighborhood_202306.db'

print("Month directory:", month_dir)
print("Save_dir directory:", save_dir)
print("ACS_file:", ACS_file)
print("home_panel file name:", home_panel_fname)

print("No dict colum file name:", no_dict_colum_fname)
print("Device home areas file name:", device_home_areas_fname)
print("Device person_per_device file name:", person_per_device_fname)


print("Device adjusted_dwelling_time_stop_CBG_fname:", adjusted_dwelling_time_stop_CBG_fname)
print("Device adjusted_dwelling_time_stop_natinol_fname:", adjusted_dwelling_time_stop_natinol_fname)

print("Restored_hourly_vistor_count_CBG file name:   ", restored_CBG_hourly_vistor_count_fname)
print("Restored_hourly_vistor_count_target file name:", restored_target_hourly_vistor_count_fname)

print("CBG_device_stop_destination_distribution file name:   ", CBG_device_stop_destination_distribution_fname)
print("CBG_device_stop_origin_distribution file name:", CBG_device_stop_origin_distribution_fname)

print("IPF_results file name:", IPF_results_fname)
print("CBG hourly population file name:", CBG_hourly_population_fname)

print("CBG_origin_outbound_distribution_sampling_rate file name:", CBG_origin_outbound_distribution_sampling_rate_fname)


Month directory: E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test_2024_home_panel_2018\2023\01\01
Save_dir directory: E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test_2024_home_panel_2018\visualization
ACS_file: E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_data\Safegraph_bias\cbg_acs_2019_county_tract_new20230929_cleaned.csv
home_panel file name: D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\Monthly_Patterns_home_panel_summary_2019_2023.zip
No dict colum file name: E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test_2024_home_panel_2018\visualization\NP_dict_column_2023_01.csv
Device home areas file name: E:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\hourly_map_test_2024_home_panel_2018\visualization\NP_split_device_home_areas_2023_01.csv


## Load monthly neighborhood patterns

In [178]:
# CBG_hourly_population_fname

In [179]:
hourly_population_map_df = pd.read_csv(CBG_hourly_population_fname, dtype={'CBG':str}).set_index('CBG')   #  = hourly_population_map_df.set_index('CBG')
hourly_population_map_df# = hourly_population_map_df.set_index('CBG') 

hour_1       hour_2       hour_3       hour_4       hour_5  \
CBG                                                                             
010010201001   852.170957   727.110156  1071.738270   609.896186   632.630787   
010010201002   872.245986  1286.382578  1499.795784  1276.437536  1163.249381   
010010202001   666.915268   628.084292   693.661513   730.122320   749.974747   
010010202002   994.474683  1006.563637   951.460485   995.970158  1020.205087   
010010203001  2806.493874  3090.551661  2581.504909  2663.602467  2708.303503   
...                   ...          ...          ...          ...          ...   
721537506011   887.213718   896.611259   900.009336   881.816119   897.987184   
721537506012  2539.671944  2575.471795  2539.152945  2517.879428  2518.848708   
721537506013  1009.726273   995.235246   997.963021   990.621432   990.693092   
721537506021  1590.063951  1584.172314  1574.505468  1575.148977  1575.499360   
721537506022   658.921272   653.239013   647.224284   647.424394   647.533351   

                   hour_6       hour_7       hour_8       hour_9      hour_10  \
CBG                                                                             
010010201001   791.622491   621.035282   592.647423   712.388183   494.194564   
010010201002  1166.269267  1151.370279  1122.288124  1357.612220  1464.983414   
010010202001   932.532982   739.849260   715.060265   677.165731   771.576197   
010010202002  1273.387498  1007.844358  1119.497739  1424.645527  1663.300325   
010010203001  2714.099482  2559.752313  2769.412165  2970.521384  2841.309407   
...                   ...          ...          ...          ...          ...   
721537506011   882.069274   881.925919   881.646096   881.218334   896.622598   
721537506012  2518.974386  2521.647723  2536.904339  2554.814471  2552.467149   
721537506013   990.702383  1005.602938   990.567064   990.430280   990.256740   
721537506021  1575.544791  1575.320652  1585.707383  1579.626449  1595.014284   
721537506022   647.547479   647.477779   647.341728   647.133749   657.494286   

                  hour_11      hour_12      hour_13      hour_14      hour_15  \
CBG                                                                             
010010201001   599.108806   541.141723   347.439347   454.108305   431.482256   
010010201002  1132.821725   916.928749  1014.385602  1000.134958   848.185565   
010010202001   584.441026   540.016411   748.416594   766.267014   646.406690   
010010202002  1297.935796   763.897619  1074.358498   958.162473  1219.478843   
010010203001  2852.557187  2668.693142  2950.972187  3706.078064  3458.937483   
...                   ...          ...          ...          ...          ...   
721537506011   893.460777   890.301475   897.833838   900.354548   894.955696   
721537506012  2553.580643  2551.411645  2559.383843  2535.737090  2535.377245   
721537506013  1005.041975   999.899487   989.794168   994.732453  1009.652245   
721537506021  1577.989918  1588.030094  1587.340393  1584.419947  1581.583807   
721537506022   646.624843   663.380065   646.166551   656.724284   650.309194   

                  hour_16      hour_17      hour_18      hour_19      hour_20  \
CBG                                                                             
010010201001   966.946292   606.509018   859.310624   380.752644  1122.911130   
010010201002   844.500840  1241.622029  1260.815993  1307.823354  1356.027949   
010010202001   875.745438   479.416410   495.776927   530.028399   571.116945   
010010202002  1060.214399  1108.906021   939.658661  1380.183852  1065.419487   
010010203001  2941.871381  3510.313088  3512.220403  2778.941671  2571.051860   
...                   ...          ...          ...          ...          ...   
721537506011   905.551554   908.222204   887.144263   898.162212   893.310372   
721537506012  2554.984202  2571.506584  2565.718606  2547.630604  2562.810256   
721537506013   994.694513  1004.662877   992.26

In [180]:
print("Each hour's population (billion):" )
hourly_population_map_df.iloc[:, 1:].sum(axis=0) / 10**9

Each hour's population (billion):


hour_2      0.327324
hour_3      0.327339
hour_4      0.327346
hour_5      0.327350
hour_6      0.327353
              ...   
hour_740    0.327333
hour_741    0.327338
hour_742    0.327348
hour_743    0.327351
hour_744    0.327359
Length: 743, dtype: float64

In [181]:
ACS_df = pd.read_csv(ACS_file)
ACS_df["CBG_FIPS"] = ACS_df["fips"].astype(str).str.zfill(12)
ACS_df["CBG"] = ACS_df["fips"].astype(str).str.zfill(12)
ACS_df['county_FIPS'] = ACS_df['CBG'].astype(str).str[:5]
print("Total US population (million):",  ACS_df['totalpopulation'].sum() / 10**6)
ACS_df

Total US population (million): 328.016242


,fips,totalpopulation,male,female,under5years,5to9years,10to14years,15to17years,18to24years,25to34years,35to44years,45to54years,55to64years,65to74years,75to84years,85yearsandover,whitealone,blackorafricanamericanalone,americanindianandalaskanativealone,asianalone,nativehawaiianandotherpacificislanderalone,someotherracealone,twoormoreraces,Hispanic_or_Latino,population25yearsandover,population25yearsandoverlessthanhighschool,population25yearsandoverhighschoolgraduateincludesequivalency,population25yearsandoversomecollege,population25yearsandoverbachelorsdegree,population25yearsandovermastersdegree,population25yearsandoverprofessionalschooldegree,population25yearsandoverdoctoratedegree,households,householdslessthan10000,households10000to14999,households15000to19999,households20000to24999,households25000to29999,households30000to34999,households35000to39999,households40000to44999,households45000to49999,households50000to59999,households60000to74999,households75000to99999,households100000to124999,households125000to149999,households150000to199999,households200000ormore,state,state_fips,county_fips,tract_fips,CBG_FIPS,CBG,county_FIPS
0,10010201001,730,293,437,17,32,24,43,104,69,87,126,96,94,23,15,613,60,0,0,0,0,57,7,510,106,187,126,41,19,31,0,315,15,52,62,10,7,7,32,9,8,4,36,19,5,7,26,16,al,1,1001,1001020100,010010201001,010010201001,01001
1,10010201002,1263,614,649,98,54,123,23,152,131,159,219,172,83,28,21,1072,92,0,2,0,0,97,19,813,60,276,209,164,99,0,5,394,11,3,38,18,5,0,2,7,10,52,42,79,19,39,47,22,al,1,1001,1001020100,010010201002,010010201002,01001
2,10010202001,835,393,442,52,89,37,78,36,79,133,76,91,90,68,6,276,507,0,0,21,0,31,30,543,68,309,114,46,6,0,0,318,38,72,40,0,21,6,20,15,12,18,27,28,16,5,0,0,al,1,1001,1001020200,010010202001,010010202001,01001
3,10010202002,1124,665,459,47,55,42,19,101,210,166,227,137,54,18,48,483,610,0,0,0,6,25,0,860,140,337,195,138,28,17,5,370,26,11,25,3,0,46,5,36,18,54,8,62,29,31,16,0,al,1,1001,1001020200,010010202002,010010202002,01001
4,10010203001,2774,1423,1351,55,258,220,80,151,505,398,336,380,216,133,42,1866,557,0,32,5,233,81,233,2010,254,736,625,204,172,0,19,1026,69,40,49,20,40,13,59,59,19,153,213,75,110,75,24,8,al,1,1001,1001020300,010010203001,010010203001,01001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220328,721537506011,883,480,403,48,78,120,12,79,45,105,70,120,169,24,13,719,0,0,0,0,164,0,883,546,167,152,126,84,17,0,0,237,102,14,45,19,0,0,31,7,19,0,0,0,0,0,0,0,pr,72,72153,72153750601,721537506011,721537506011,72153
220329,721537506012,2523,1010,1513,227,175,96,36,308,250,260,436,197,394,106,38,1946,89,51,0,0,408,29,2523,1681,190,675,414,291,77,15,19,740,97,140,96,61,97,58,32,28,47,57,8,0,19,0,0,0,pr,72,72153,72153750601,721537506012,721537506012,72153
220330,721537506013,991,477,514,86,37,162,76,190,53,90,191,89,17,0,0,538,227,0,0,0,226,0,956,440,0,30,74,290,0,18,28,256,48,9,8,0,42,0,0,39,0,12,56,24,18,0,0,0,pr,72,72153,72153750601,721537506013,721537506013,72153
220331,721537506021,1577,795,782,68,36,18,24,82,169,225,272,254,232,188,9,1116,23,0,0,0,438,0,1577,1349,590,482,64,177,36,0,0,560,303,78,91,8,0,11,32,0,0,10,27,0,0,0,0,0,pr,72,72153,72153750602,721537506021,721537506021,72153


## Show hourly population map

In [182]:
# Load data
hourly_population_map_df

hour_1       hour_2       hour_3       hour_4       hour_5  \
CBG                                                                             
010010201001   852.170957   727.110156  1071.738270   609.896186   632.630787   
010010201002   872.245986  1286.382578  1499.795784  1276.437536  1163.249381   
010010202001   666.915268   628.084292   693.661513   730.122320   749.974747   
010010202002   994.474683  1006.563637   951.460485   995.970158  1020.205087   
010010203001  2806.493874  3090.551661  2581.504909  2663.602467  2708.303503   
...                   ...          ...          ...          ...          ...   
721537506011   887.213718   896.611259   900.009336   881.816119   897.987184   
721537506012  2539.671944  2575.471795  2539.152945  2517.879428  2518.848708   
721537506013  1009.726273   995.235246   997.963021   990.621432   990.693092   
721537506021  1590.063951  1584.172314  1574.505468  1575.148977  1575.499360   
721537506022   658.921272   653.239013   647.224284   647.424394   647.533351   

                   hour_6       hour_7       hour_8       hour_9      hour_10  \
CBG                                                                             
010010201001   791.622491   621.035282   592.647423   712.388183   494.194564   
010010201002  1166.269267  1151.370279  1122.288124  1357.612220  1464.983414   
010010202001   932.532982   739.849260   715.060265   677.165731   771.576197   
010010202002  1273.387498  1007.844358  1119.497739  1424.645527  1663.300325   
010010203001  2714.099482  2559.752313  2769.412165  2970.521384  2841.309407   
...                   ...          ...          ...          ...          ...   
721537506011   882.069274   881.925919   881.646096   881.218334   896.622598   
721537506012  2518.974386  2521.647723  2536.904339  2554.814471  2552.467149   
721537506013   990.702383  1005.602938   990.567064   990.430280   990.256740   
721537506021  1575.544791  1575.320652  1585.707383  1579.626449  1595.014284   
721537506022   647.547479   647.477779   647.341728   647.133749   657.494286   

                  hour_11      hour_12      hour_13      hour_14      hour_15  \
CBG                                                                             
010010201001   599.108806   541.141723   347.439347   454.108305   431.482256   
010010201002  1132.821725   916.928749  1014.385602  1000.134958   848.185565   
010010202001   584.441026   540.016411   748.416594   766.267014   646.406690   
010010202002  1297.935796   763.897619  1074.358498   958.162473  1219.478843   
010010203001  2852.557187  2668.693142  2950.972187  3706.078064  3458.937483   
...                   ...          ...          ...          ...          ...   
721537506011   893.460777   890.301475   897.833838   900.354548   894.955696   
721537506012  2553.580643  2551.411645  2559.383843  2535.737090  2535.377245   
721537506013  1005.041975   999.899487   989.794168   994.732453  1009.652245   
721537506021  1577.989918  1588.030094  1587.340393  1584.419947  1581.583807   
721537506022   646.624843   663.380065   646.166551   656.724284   650.309194   

                  hour_16      hour_17      hour_18      hour_19      hour_20  \
CBG                                                                             
010010201001   966.946292   606.509018   859.310624   380.752644  1122.911130   
010010201002   844.500840  1241.622029  1260.815993  1307.823354  1356.027949   
010010202001   875.745438   479.416410   495.776927   530.028399   571.116945   
010010202002  1060.214399  1108.906021   939.658661  1380.183852  1065.419487   
010010203001  2941.871381  3510.313088  3512.220403  2778.941671  2571.051860   
...                   ...          ...          ...          ...          ...   
721537506011   905.551554   908.222204   887.144263   898.162212   893.310372   
721537506012  2554.984202  2571.506584  2565.718606  2547.630604  2562.810256   
721537506013   994.694513  1004.662877   992.26

In [183]:
hour_cnt = hourly_population_map_df.shape[1] - 1
print("hour count:", hour_cnt)

hour count: 743


In [184]:
def get_county_FIPS_dict():
    county_gdf = gpd.read_file(r'https://github.com/gladcolor/spatial_data/raw/master/cb_2019_us_county_20m.zip')
    # county_gdf = gpd.read_file(r"D:\OneDrive_PSU\OneDrive - The Pennsylvania State University\Research_doc\Wild_fire\vectors\cb_2019_us_county_20m.zip")
    state_gdf = gpd.read_file(r'https://github.com/gladcolor/spatial_data/raw/master/cb_2019_us_state_20m.zip')
    county_gdf = county_gdf.merge(state_gdf[['STATEFP', 'STUSPS', 'NAME']], left_on='STATEFP', right_on='STATEFP', suffixes=['_county', '_state'])
    county_gdf = county_gdf.sort_values('GEOID')
    county_gdf['county_state'] = county_gdf['NAME_county'] + ', ' + county_gdf['STUSPS']
    county_gdf['GEOID'] = county_gdf['GEOID'].astype(str).str.zfill(5)
    county_FIPS_dict = county_gdf.set_index(['county_state'])['GEOID'].to_dict()
    county_df = county_gdf[['NAME_state', 'county_state', 'GEOID']]
    return county_FIPS_dict, county_df

county_FIPS_dict, county_df= get_county_FIPS_dict()

list(county_FIPS_dict.items())[0], list(county_FIPS_dict.items())[-1]
# county_FIPS_dict
county_df

,NAME_state,county_state,GEOID
2483,Alabama,"Autauga, AL",01001
2494,Alabama,"Baldwin, AL",01003
2512,Alabama,"Barbour, AL",01005
2506,Alabama,"Bibb, AL",01007
2471,Alabama,"Blount, AL",01009
...,...,...,...
1910,Puerto Rico,"Vega Baja, PR",72145
1919,Puerto Rico,"Vieques, PR",72147
1882,Puerto Rico,"Villalba, PR",72149
1876,Puerto Rico,"Yabucoa, PR",72151


In [185]:
# type(county_gdf)

In [186]:
# CBG2019_gdf = gpd.read_file(CBG_2019_fname)
# CBG2019_gdf['county_FIPS'] = CBG2019_gdf['GEOID'].astype(str).str.zfill(12).str[:5]
# CBG2019_gdf['CBG'] = CBG2019_gdf['GEOID'].astype(str).str.zfill(12) 
# CBG2019_gdf

In [187]:
hourly_population_map_df.iloc[:, :hour_cnt + 1]

hour_1       hour_2       hour_3       hour_4       hour_5  \
CBG                                                                             
010010201001   852.170957   727.110156  1071.738270   609.896186   632.630787   
010010201002   872.245986  1286.382578  1499.795784  1276.437536  1163.249381   
010010202001   666.915268   628.084292   693.661513   730.122320   749.974747   
010010202002   994.474683  1006.563637   951.460485   995.970158  1020.205087   
010010203001  2806.493874  3090.551661  2581.504909  2663.602467  2708.303503   
...                   ...          ...          ...          ...          ...   
721537506011   887.213718   896.611259   900.009336   881.816119   897.987184   
721537506012  2539.671944  2575.471795  2539.152945  2517.879428  2518.848708   
721537506013  1009.726273   995.235246   997.963021   990.621432   990.693092   
721537506021  1590.063951  1584.172314  1574.505468  1575.148977  1575.499360   
721537506022   658.921272   653.239013   647.224284   647.424394   647.533351   

                   hour_6       hour_7       hour_8       hour_9      hour_10  \
CBG                                                                             
010010201001   791.622491   621.035282   592.647423   712.388183   494.194564   
010010201002  1166.269267  1151.370279  1122.288124  1357.612220  1464.983414   
010010202001   932.532982   739.849260   715.060265   677.165731   771.576197   
010010202002  1273.387498  1007.844358  1119.497739  1424.645527  1663.300325   
010010203001  2714.099482  2559.752313  2769.412165  2970.521384  2841.309407   
...                   ...          ...          ...          ...          ...   
721537506011   882.069274   881.925919   881.646096   881.218334   896.622598   
721537506012  2518.974386  2521.647723  2536.904339  2554.814471  2552.467149   
721537506013   990.702383  1005.602938   990.567064   990.430280   990.256740   
721537506021  1575.544791  1575.320652  1585.707383  1579.626449  1595.014284   
721537506022   647.547479   647.477779   647.341728   647.133749   657.494286   

                  hour_11      hour_12      hour_13      hour_14      hour_15  \
CBG                                                                             
010010201001   599.108806   541.141723   347.439347   454.108305   431.482256   
010010201002  1132.821725   916.928749  1014.385602  1000.134958   848.185565   
010010202001   584.441026   540.016411   748.416594   766.267014   646.406690   
010010202002  1297.935796   763.897619  1074.358498   958.162473  1219.478843   
010010203001  2852.557187  2668.693142  2950.972187  3706.078064  3458.937483   
...                   ...          ...          ...          ...          ...   
721537506011   893.460777   890.301475   897.833838   900.354548   894.955696   
721537506012  2553.580643  2551.411645  2559.383843  2535.737090  2535.377245   
721537506013  1005.041975   999.899487   989.794168   994.732453  1009.652245   
721537506021  1577.989918  1588.030094  1587.340393  1584.419947  1581.583807   
721537506022   646.624843   663.380065   646.166551   656.724284   650.309194   

                  hour_16      hour_17      hour_18      hour_19      hour_20  \
CBG                                                                             
010010201001   966.946292   606.509018   859.310624   380.752644  1122.911130   
010010201002   844.500840  1241.622029  1260.815993  1307.823354  1356.027949   
010010202001   875.745438   479.416410   495.776927   530.028399   571.116945   
010010202002  1060.214399  1108.906021   939.658661  1380.183852  1065.419487   
010010203001  2941.871381  3510.313088  3512.220403  2778.941671  2571.051860   
...                   ...          ...          ...          ...          ...   
721537506011   905.551554   908.222204   887.144263   898.162212   893.310372   
721537506012  2554.984202  2571.506584  2565.718606  2547.630604  2562.810256   
721537506013   994.694513  1004.662877   992.26

In [188]:
CBG_hourly_outbound_df = pd.read_csv(IPF_results_fname, dtype={'CBG':str}).set_index('CBG').iloc[:, -hour_cnt-1:]
CBG_hourly_outbound_df# .iloc[:, 1:]

,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,hour_24,hour_25,hour_26,hour_27,hour_28,hour_29,hour_30,hour_31,hour_32,hour_33,hour_34,hour_35,hour_36,hour_37,hour_38,hour_39,hour_40,hour_41,hour_42,hour_43,hour_44,hour_45,hour_46,hour_47,hour_48,hour_49,hour_50,hour_51,hour_52,hour_53,hour_54,hour_55,hour_56,hour_57,hour_58,hour_59,hour_60,hour_61,hour_62,hour_63,hour_64,hour_65,hour_66,hour_67,hour_68,hour_69,hour_70,hour_71,hour_72,hour_73,hour_74,hour_75,hour_76,hour_77,hour_78,hour_79,hour_80,hour_81,hour_82,hour_83,hour_84,hour_85,hour_86,hour_87,hour_88,hour_89,hour_90,hour_91,hour_92,hour_93,hour_94,hour_95,hour_96,hour_97,hour_98,hour_99,hour_100,hour_101,hour_102,hour_103,hour_104,hour_105,hour_106,hour_107,hour_108,hour_109,hour_110,hour_111,hour_112,hour_113,hour_114,hour_115,hour_116,hour_117,hour_118,hour_119,hour_120,hour_121,hour_122,hour_123,hour_124,hour_125,hour_126,hour_127,hour_128,hour_129,hour_130,hour_131,hour_132,hour_133,hour_134,hour_135,hour_136,hour_137,hour_138,hour_139,hour_140,hour_141,hour_142,hour_143,hour_144,hour_145,hour_146,hour_147,hour_148,hour_149,hour_150,hour_151,hour_152,hour_153,hour_154,hour_155,hour_156,hour_157,hour_158,hour_159,hour_160,hour_161,hour_162,hour_163,hour_164,hour_165,hour_166,hour_167,hour_168,hour_169,hour_170,hour_171,hour_172,hour_173,hour_174,hour_175,hour_176,hour_177,hour_178,hour_179,hour_180,hour_181,hour_182,hour_183,hour_184,hour_185,hour_186,hour_187,hour_188,hour_189,hour_190,hour_191,hour_192,hour_193,hour_194,hour_195,hour_196,hour_197,hour_198,hour_199,hour_200,hour_201,hour_202,hour_203,hour_204,hour_205,hour_206,hour_207,hour_208,hour_209,hour_210,hour_211,hour_212,hour_213,hour_214,hour_215,hour_216,hour_217,hour_218,hour_219,hour_220,hour_221,hour_222,hour_223,hour_224,hour_225,hour_226,hour_227,hour_228,hour_229,hour_230,hour_231,hour_232,hour_233,hour_234,hour_235,hour_236,hour_237,hour_238,hour_239,hour_240,hour_241,hour_242,hour_243,hour_244,hour_245,hour_246,hour_247,hour_248,hour_249,hour_250,hour_251,hour_252,hour_253,hour_254,hour_255,hour_256,hour_257,hour_258,hour_259,hour_260,hour_261,hour_262,hour_263,hour_264,hour_265,hour_266,hour_267,hour_268,hour_269,hour_270,hour_271,hour_272,hour_273,hour_274,hour_275,hour_276,hour_277,hour_278,hour_279,hour_280,hour_281,hour_282,hour_283,hour_284,hour_285,hour_286,hour_287,hour_288,hour_289,hour_290,hour_291,hour_292,hour_293,hour_294,hour_295,hour_296,hour_297,hour_298,hour_299,hour_300,hour_301,hour_302,hour_303,hour_304,hour_305,hour_306,hour_307,hour_308,hour_309,hour_310,hour_311,hour_312,hour_313,hour_314,hour_315,hour_316,hour_317,hour_318,hour_319,hour_320,hour_321,hour_322,hour_323,hour_324,hour_325,hour_326,hour_327,hour_328,hour_329,hour_330,hour_331,hour_332,hour_333,hour_334,hour_335,hour_336,hour_337,hour_338,hour_339,hour_340,hour_341,hour_342,hour_343,hour_344,hour_345,hour_346,hour_347,hour_348,hour_349,hour_350,hour_351,hour_352,hour_353,hour_354,hour_355,hour_356,hour_357,hour_358,hour_359,hour_360,hour_361,hour_362,hour_363,hour_364,hour_365,hour_366,hour_367,hour_368,hour_369,hour_370,hour_371,hour_372,hour_373,hour_374,hour_375,hour_376,hour_377,hour_378,hour_379,hour_380,hour_381,hour_382,hour_383,hour_384,hour_385,hour_386,hour_387,hour_388,hour_389,hour_390,hour_391,hour_392,hour_393,hour_394,hour_395,hour_396,hour_397,hour_398,hour_399,hour_400,hour_401,hour_402,hour_403,hour_404,hour_405,hour_406,hour_407,hour_408,hour_409,hour_410,hour_411,hour_412,hour_413,hour_414,hour_415,hour_416,hour_417,hour_418,hour_419,hour_420,hour_421,hour_422,hour_423,hour_424,hour_425,hour_426,hour_427,hour_428,hour_429,hour_430,hour_431,hour_432,hour_433,hour_434,hour_435,hour_436,hour_437,hour_438,hour_439,hour_440,hour_441,hour_442,hour_443,hour_444,hour_445,hour_446,hour_447,hour_448,hour_449,hour_450,hour_451,hour_452,hour_453,hour_454,hour_455,hour_456,hou

In [189]:
CBG_hourly_inbound_df = pd.read_csv(inbound_hourly_df_fname, dtype={'CBG':str}).set_index('CBG').iloc[:, :hour_cnt + 1]
CBG_hourly_inbound_df

MemoryError: Unable to allocate 1.23 GiB for an array with shape (746, 220449) and data type float64

In [ ]:
# CBG_hourly_outbound_df

In [ ]:
# hourly outbound
CBG_hourly_outbound_df['county_FIPS'] = CBG_hourly_outbound_df.reset_index()['CBG'].astype(str).str[:5].to_list()
target_hourly_outbound_df = CBG_hourly_outbound_df[CBG_hourly_outbound_df['county_FIPS'].isin(['36061'] )]
target_hourly_outbound_df = target_hourly_outbound_df.drop(columns=['county_FIPS'])
target_hourly_outbound_df

In [ ]:
MTA_traffic_df = pd.read_csv(NYC_MTA_fname)
MTA_traffic_df['Date'] = MTA_traffic_df['Date'] + ' 13:00'
MTA_traffic_df['Date'] = pd.to_datetime(MTA_traffic_df['Date'])
MTA_traffic_df.set_index('Date', inplace=True)

MTA_traffic_df = MTA_traffic_df[(MTA_traffic_df.index.year == year) & (MTA_traffic_df.index.month == month)]
# MTA_traffic_df['Subways: Total Estimated Ridership'].plot()

In [ ]:
import warnings

# Suppress all future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

cmap = 'viridis'
# day = 18

target_county = 'New York, NY'
target_county = 'Centre, PA'
target_county = 'Richland, SC'
target_county = 'Lexington, SC'
target_county = 'Union, NJ'
target_county = 'Fairfax, VA'
target_county = 'Los Angeles, CA'
target_county = 'Cook, IL'
target_county = 'Harris, TX'
target_county = 'Maricopa, AZ'
target_county = 'San Diego, CA'
target_county = 'Orange, CA'
target_county = 'Miami-Dade, FL'
target_county = 'Dallas, TX'
target_county = 'King, WA'
target_county = 'Riverside, CA'
target_county = 'Hudson, NJ'
target_county = 'Essex, NJ'
target_county = 'Berkshire, MA'



def draw_county_map(target_county):
    
    county_FIPS = county_FIPS_dict[target_county]
    
    target_counties = [county_FIPS] # ['36061']   # 06039

    # hourly population
    hourly_population_map_df['county_FIPS'] = hourly_population_map_df.reset_index()['CBG'].astype(str).str[:5].to_list()
    target_hourly_popu_df = hourly_population_map_df[hourly_population_map_df['county_FIPS'].isin(target_counties)]
    target_hourly_popu_df = target_hourly_popu_df.drop(columns=['county_FIPS'])
    target_hourly_county_popu_df = pd.DataFrame(target_hourly_popu_df.sum(axis=0), columns=['hourly_population'])

    # hourly inbound
    CBG_hourly_inbound_df['county_FIPS'] = CBG_hourly_inbound_df.reset_index()['CBG'].astype(str).str[:5].to_list()
    target_hourly_inbound_df = CBG_hourly_inbound_df[CBG_hourly_inbound_df['county_FIPS'].isin(target_counties)]
    target_hourly_inbound_df = target_hourly_inbound_df.drop(columns=['county_FIPS'])
    target_hourly_county_inbound_df = pd.DataFrame(target_hourly_inbound_df.sum(axis=0), columns=['hourly_inbound'])

    # hourly outbound
    CBG_hourly_outbound_df['county_FIPS'] = CBG_hourly_outbound_df.reset_index()['CBG'].astype(str).str[:5].to_list()
    target_hourly_outbound_df = CBG_hourly_outbound_df[CBG_hourly_outbound_df['county_FIPS'].isin(target_counties)]
    target_hourly_outbound_df = target_hourly_outbound_df.drop(columns=['county_FIPS'])
    target_hourly_county_outbound_df = pd.DataFrame(target_hourly_outbound_df.sum(axis=0), columns=['hourly_outbound'])
    
    # print("target_hourly_county_popu_df.index:", list(target_hourly_county_popu_df.index))
    # print("target_hourly_county_outbound_df.index:", list(target_hourly_county_outbound_df.index))
    # print("target_hourly_county_inbound_df.index:",  list(target_hourly_county_inbound_df.index))
     

    # print(target_hourly_county_popu_df)

    
    
    # target_blockgroup_gdf = CBG2019_gdf[CBG2019_gdf['county_FIPS'].isin(target_counties)]
    # target_blockgroup_gdf
    
    # row_cnt = 2
    # col_cnt = 13
    # fig, axs = plt.subplots(figsize=(40, 10), nrows=row_cnt, ncols=col_cnt)
    fig, ax = plt.subplots(figsize=(40, 20))

    # Generate hourly list
    start_date = pd.to_datetime(f'{year}-{month:02d}-01')
    next_month = month + 1
    # print("month, next_month", month, next_month, type(year), type(month))
    _, last_month_day = calendar.monthrange(year, month)
    # end_date = f"{year}-{month:02d}-{last_month_day:02d}"
    
    #
    end_date = f"{year}-{next_month:02d}-01"
    
    hourly_list = pd.date_range(start_date, end_date, freq='H')[:-1]
    # print(month, hourly_list)
    
    
    county_ACS_popu_df = ACS_df[ACS_df['county_FIPS'].isin(target_counties)] 
    ACS_popu = county_ACS_popu_df['totalpopulation'].sum()
    
    target_hourly_county_popu_df.index = hourly_list 
    target_hourly_county_inbound_df.index = hourly_list 
    target_hourly_county_outbound_df.index = hourly_list
    
    # target_hourly_county_popu_df.plot(ax=ax)
    # ax.fill_between(x=hourly_list, y1=ACS_popu, y2=target_hourly_county_inbound_df['hourly_inbound'].to_list() + ACS_popu, alpha=0.3)
    # ax.fill_between(x=hourly_list, y1=ACS_popu, y2=((target_hourly_county_outbound_df['hourly_outbound'] * -1).to_list()) + ACS_popu, alpha=0.3, label="Outbound")
    # (target_hourly_county_inbound_df + ACS_popu).plot.area(ax=ax)
    # ((-target_hourly_county_outbound_df + ACS_popu)).plot.area(ax=ax)
     
 
    # MTA_traffic_df['Subways: Total Estimated Ridership'].plot(ax=ax)
    # ax.plot(MTA_traffic_df.index, MTA_traffic_df['Subways: Total Estimated Ridership'], label='Subways: Total Estimated Ridership')
    # plt.plot(daily_df.index, daily_df['data_column'], label='Daily Data', marker='o')
    target_hourly_county_popu_df.plot(ax=ax, color='orange')
    ax.fill_between(x=hourly_list, y1=0, y2=target_hourly_county_inbound_df['hourly_inbound'].to_list()  , alpha=0.15, label="Inbound")
    ax.fill_between(x=hourly_list, y1=0, y2=((target_hourly_county_outbound_df['hourly_outbound'] * -1).to_list())  , alpha=0.15, label="Outbound")

    print("target_counties:", target_counties)
    # print("county_ACS_popu_df:", county_ACS_popu_df)
    ax.axhline(ACS_popu, color='red', label='ACS population')
    ax.legend()   

    png_name = os.path.join(save_dir, f"{target_county.replace(', ', '_')}_hourly_population_{year}{month:02}.png")
    
    print("PNG name:", png_name)
    
    plt.savefig(png_name, dpi=300, bbox_inches='tight', pad_inches=0.2)
    # plt.close()


target_county_list = ['New York, NY']
# target_county_list = ['Orange, CA']
# target_county_list = ['Lexington, SC']
# target_county_list = ['Richland, SC']

mask = county_df['county_state'].isin(target_county_list)
target_county_df = county_df[mask]
target_county_dict = target_county_df.set_index(['county_state'])['GEOID'].to_dict()

for target_county, FIPS in tqdm(target_county_dict.items()):
# for target_county, FIPS in tqdm(county_FIPS_dict.items()):
    print(target_county)
    draw_county_map(target_county)
    
    break

print("Done")